In [64]:
import pandas as pd

In [65]:
import keras

In [66]:
!ls /data


fake-news.zip			   glove.6B.100d.txt  glove.6B.zip
fake.csv			   glove.6B.200d.txt  train_stances.csv
fake_or_real_news.csv		   glove.6B.300d.txt
fake_real_news_dataset-master.zip  glove.6B.50d.txt


In [67]:
df1 = pd.read_csv('/data/fake.csv')

In [68]:
df1.head()

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


In [69]:
df1 = df1[(df1.language == 'english') & (df1.type == 'fake')]


In [70]:
df1 = df1[['text', 'type']]
df1 = df1.rename(columns={'type': 'label'})

In [71]:
df2 = pd.read_csv('/data/fake_or_real_news.csv')

In [72]:
df2 = df2[['text', 'label']]

In [73]:
df2['label'] = df2['label'].map({'FAKE': 'fake', 'REAL': 'real'})

In [74]:
df = pd.concat([df1, df2], axis = 0, ignore_index=True)

In [75]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [76]:
MAX_NB_WORDS = 20000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)

In [77]:
df.columns

Index(['text', 'label'], dtype='object')

In [78]:
tokenizer.fit_on_texts(df.text)

In [79]:
sequences = tokenizer.texts_to_sequences(df.text)

In [80]:
word_index = tokenizer.word_index
print('Found %s unique tokes. ' % len(word_index))

Found 98886 unique tokes. 


In [83]:
MAX_SEQUENCE_LENGTH = 1000
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
data.shape


(6354, 1000)

In [84]:
import numpy as np
df['label'] = df['label'].map({'fake': 1, 'real': 0})
# print(df.label)
labels = to_categorical(df.label, num_classes=2)
type(labels)
labels.shape

(6354, 2)

In [85]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', df.label.shape)

Shape of data tensor: (6354, 1000)
Shape of label tensor: (6354,)


In [86]:
VALIDATION_SPLIT = 0.2
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [87]:
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [88]:
import os
import numpy as np
GLOVE_DIR = '/data'
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'),encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [89]:
print('Preparing embedding matrix.')
EMBEDDING_DIM = 100
num_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

Preparing embedding matrix.


In [90]:
from keras.layers import Embedding

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [166]:
MAX_SEQUENCE_LENGTH = 1000
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

In [172]:
MAX_SEQUENCE_LENGTH = 1000
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Dense(128, activation='relu')(embedded_sequences)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

In [173]:
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras import optimizers
model = Model(sequence_input, preds)
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [178]:
model.fit(x_train, y_train,
          batch_size=800,
          epochs=20,
          validation_data=(x_val, y_val)
          )

Train on 5084 samples, validate on 1270 samples
Epoch 1/20
5084/5084 [==============================] - 20s - loss: 0.0185 - acc: 0.9996 - val_loss: 0.5107 - val_acc: 0.8173
Epoch 2/20
5084/5084 [==============================] - 20s - loss: 0.0091 - acc: 0.9996 - val_loss: 0.5478 - val_acc: 0.8228
Epoch 3/20
5084/5084 [==============================] - 20s - loss: 0.0058 - acc: 0.9996 - val_loss: 0.5769 - val_acc: 0.8220
Epoch 4/20
5084/5084 [==============================] - 20s - loss: 0.0040 - acc: 0.9996 - val_loss: 0.6209 - val_acc: 0.8205
Epoch 5/20
5084/5084 [==============================] - 20s - loss: 0.0027 - acc: 0.9998 - val_loss: 0.6455 - val_acc: 0.8236
Epoch 6/20
5084/5084 [==============================] - 20s - loss: 0.0018 - acc: 1.0000 - val_loss: 0.6912 - val_acc: 0.8197
Epoch 7/20
5084/5084 [==============================] - 20s - loss: 0.0014 - acc: 1.0000 - val_loss: 0.7244 - val_acc: 0.8189
Epoch 8/20
5084/5084 [==============================] - 20s - loss: 7.

In [177]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 1000)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 1000, 100)         2000000   
_________________________________________________________________
dense_23 (Dense)             (None, 1000, 128)         12928     
_________________________________________________________________
dense_24 (Dense)             (None, 1000, 128)         16512     
_________________________________________________________________
dense_25 (Dense)             (None, 1000, 128)         16512     
_________________________________________________________________
flatten_16 (Flatten)         (None, 128000)            0         
_________________________________________________________________
dense_26 (Dense)             (None, 128)               16384128  
__________

In [179]:
from sklearn.metrics import confusion_matrix
y_prob = model.predict(x_val)
y_pred = np.argmax(y_prob,axis=-1)
confusion_matrix(df.label[-num_validation_samples:], y_pred)
# df.label[-num_validation_samples:]




array([[286, 338],
       [323, 323]])

In [149]:
model.compile??

In [180]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.